In [1]:
# Arguments
REWARD = -0.1 # constant reward for non-terminal states
DISCOUNT = 0.7
MAX_ERROR = 10**(-3)

In [2]:
import random

In [3]:
# Set up the initial environment
NUM_ACTIONS = 4
ACTIONS = [(1, 0), (0, -1), (-1, 0), (0, 1)] # Down, Left, Up, Right
NUM_ROW = 4
NUM_COL = 4
U = [[0, 0, 0, -2], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 2]]
policy = [[random.randint(1, 3) for j in range(NUM_COL)] for i in range(NUM_ROW)] # construct a random policy

In [4]:
# Visualization
def printEnvironment(arr, policy=False):
    res = ""
    for r in range(NUM_ROW):
        res += "|"
        for c in range(NUM_COL):
            if r ==1 and c == 2:
                val = "WALL"
            elif r ==0  and c == 3:
                val = "-2"    
            elif r == 3 and c==3:
                val = "2"
            else:
                val = ["Down", "Left", "Up", "Right"][arr[r][c]]
            res += " " + val[:5].ljust(5) + " |" # format
        res += "\n"
    print(res)

In [5]:
# Get the utility of the state reached by performing the given action from the given state
def getU(U, r, c, action):
    dr, dc = ACTIONS[action]
    newR, newC = r+dr, c+dc
    if newR < 0 or newC < 0 or newR >= NUM_ROW or newC >= NUM_COL or (newR == newC == 1): # collide with the boundary or the wall
        return U[r][c]
    else:
        return U[newR][newC]

In [6]:
# Calculate the utility of a state given an action
def calculateU(U, r, c, action):
    u = REWARD
    u += 0.2 * DISCOUNT * getU(U, r, c, (action-1)%4)
    u += 0.6 * DISCOUNT * getU(U, r, c, action)
    u += 0.2 * DISCOUNT * getU(U, r, c, (action+1)%4)
    return u

In [7]:
# Perform some simplified value iteration steps to get an approximation of the utilities
def policyEvaluation(policy, U):
    while True:
        nextU = [[0, 0, 0, -2], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 2]]
        error = 0
        for r in range(NUM_ROW):
            for c in range(NUM_COL):
                if (r == 0 and c == 3) or (r == 3 and c == 3) or (r == 1 and c == 2):
                    continue
                nextU[r][c] = calculateU(U, r, c, policy[r][c]) # simplified Bellman update
                error = max(error, abs(nextU[r][c]-U[r][c]))
        U = nextU
        if error < MAX_ERROR * (1-DISCOUNT) / DISCOUNT:
            break
    return U

In [8]:
def policyIteration(policy, U):
    print("During the policy iteration:\n")
    while True:
        U = policyEvaluation(policy, U)
        unchanged = True
        for r in range(NUM_ROW):
            for c in range(NUM_COL):
                if (r == 0 and c == 3) or (r == 3 and c == 3) or (r == 1 and c == 2):
                    continue
                maxAction, maxU = None, -float("inf")
                for action in range(NUM_ACTIONS):
                    u = calculateU(U, r, c, action)
                    if u > maxU:
                        maxAction, maxU = action, u
                if maxU > calculateU(U, r, c, policy[r][c]):
                    policy[r][c] = maxAction # the action that maximizes the utility
                    unchanged = False
        if unchanged:
            break
        printEnvironment(policy)
    return policy

In [9]:
# Print the initial environment
print("The initial random policy is:\n")
printEnvironment(policy)

The initial random policy is:

| Right | Up    | Right | -2    |
| Up    | Up    | WALL  | Left  |
| Right | Up    | Left  | Up    |
| Up    | Right | Left  | 2     |



In [10]:
# Policy iteration
policy = policyIteration(policy, U)

During the policy iteration:

| Left  | Left  | Down  | -2    |
| Down  | Right | WALL  | Down  |
| Down  | Right | Up    | Down  |
| Right | Right | Right | 2     |

| Down  | Left  | Left  | -2    |
| Down  | Right | WALL  | Down  |
| Down  | Down  | Right | Down  |
| Right | Right | Right | 2     |

| Down  | Left  | Left  | -2    |
| Down  | Down  | WALL  | Down  |
| Down  | Right | Down  | Down  |
| Right | Right | Right | 2     |

| Down  | Left  | Left  | -2    |
| Down  | Down  | WALL  | Down  |
| Right | Right | Down  | Down  |
| Right | Right | Right | 2     |



In [11]:
# Print the optimal policy
print("The optimal policy is:\n")
printEnvironment(policy)

The optimal policy is:

| Down  | Left  | Left  | -2    |
| Down  | Down  | WALL  | Down  |
| Right | Right | Down  | Down  |
| Right | Right | Right | 2     |

